# Imports

In [19]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch

#Neural network libraries
import torch.nn as nn
import torch.nn.functional as F

# #Creating optimizer
import torch.optim as optim

# For terminating program (useful for stopping before training is complete and seeing entire result)
import signal
import sys, os

# CSV Management

In [20]:
script_dir = os.path.abspath('') # absolute file path in Jupyter
file_path = os.path.join(script_dir, "AI EarthHack Dataset.csv") # file path for the text file input

import_dataset = pd.read_csv(file_path, header=None)

dataset = []
for i in range(len(import_dataset[0])):
    dataset.append([import_dataset[0][i], import_dataset[1][i], import_dataset[2][i]])

print(dataset[150])


['150', 'Recycling and reusing materials from discarded products can be challenging. One of the main challenges is the complexity of many modern products, which can be made up of a large number of different materials and components. This can make it difficult to separate and process these materials in an efficient and cost-effective manner. \n', 'Napapijri has developed a line of high-performance jackets, launched under the Circular Series, that are made entirely from a single polymer called Nylon 6, made from recycled fish nets. The material is durable, high performing, and easily recycled, making it suitable for use in the circular economy. Napapijri has simplified the design of the jackets so that all components, including the fabric, filling, and trimmings, are made from the same material. This makes it easier to recycle the jackets at the end of their lifespan. \nCustomers who purchase a Circular Series jacket are encouraged to register their product online and can receive a 20% d